In [689]:
%pip install --upgrade pip
%pip install pandas numpy openpyxl seaborn matplotlib

import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns

train_data = pd.read_excel("Data_Train.xlsx")
test_data = pd.read_excel("Test_set.xlsx")

y = train_data['Price']

train_data = train_data.drop('Price', axis=1)

train_data['is_train'] = 1
test_data['is_train'] = 0


df = pd.concat([train_data, test_data], ignore_index=True)

print(f"Combined dataset shape: {df.shape}")
print(f"Training samples: {df['is_train'].sum()}")
print(f"Test samples: {(df['is_train'] == 0).sum()}")
print("\nFirst few rows of combined dataset:")
df.head()

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Combined dataset shape: (13354, 11)
Training samples: 10683
Test samples: 2671

First few rows of combined dataset:


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,is_train
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,1
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,1
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,1
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,1
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,1


In [690]:
df.shape

(13354, 11)

In [691]:
df.drop('Route', axis=1, inplace=True)

In [692]:
df.isnull().sum()


Airline            0
Date_of_Journey    0
Source             0
Destination        0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
is_train           0
dtype: int64

In [693]:
rows_with_nulls = df[df.isnull().any(axis=1)]
rows_with_nulls


,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,is_train
9039,Air India,6/05/2019,Delhi,Cochin,09:45,09:25 07 May,23h 40m,NaN,No info,1


In [694]:

df.shape

(13354, 10)

In [695]:
df['Additional_Info'].value_counts()

Additional_Info
No info                         10493
In-flight meal not included      2426
No check-in baggage included      396
1 Long layover                     20
Change airports                     8
Business class                      5
No Info                             3
1 Short layover                     1
Red-eye flight                      1
2 Long layover                      1
Name: count, dtype: int64

In [696]:
df['Additional_Info'] = df['Additional_Info'].apply(lambda x: 'No info' if x == 'No info' else 'Others')
df['Additional_Info'].value_counts()


Additional_Info
No info    10493
Others      2861
Name: count, dtype: int64

In [697]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          13354 non-null  object
 1   Date_of_Journey  13354 non-null  object
 2   Source           13354 non-null  object
 3   Destination      13354 non-null  object
 4   Dep_Time         13354 non-null  object
 5   Arrival_Time     13354 non-null  object
 6   Duration         13354 non-null  object
 7   Total_Stops      13353 non-null  object
 8   Additional_Info  13354 non-null  object
 9   is_train         13354 non-null  int64 
dtypes: int64(1), object(9)
memory usage: 1.0+ MB


In [698]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,is_train
0,IndiGo,24/03/2019,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,1
1,Air India,1/05/2019,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,1
2,Jet Airways,9/06/2019,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,1
3,IndiGo,12/05/2019,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,1
4,IndiGo,01/03/2019,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,1


In [699]:
#feature engineering 
df['Date'] = df['Date_of_Journey'].str.split('/').str[0]
df['Month'] = df['Date_of_Journey'].str.split('/').str[1]
df['Year'] = df['Date_of_Journey'].str.split('/').str[2]
df.drop('Date_of_Journey',axis=1,inplace=True)
df.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,is_train,Date,Month,Year
0,IndiGo,Banglore,New Delhi,22:20,01:10 22 Mar,2h 50m,non-stop,No info,1,24,03,2019
1,Air India,Kolkata,Banglore,05:50,13:15,7h 25m,2 stops,No info,1,1,05,2019
2,Jet Airways,Delhi,Cochin,09:25,04:25 10 Jun,19h,2 stops,No info,1,9,06,2019
3,IndiGo,Kolkata,Banglore,18:05,23:30,5h 25m,1 stop,No info,1,12,05,2019
4,IndiGo,Banglore,New Delhi,16:50,21:35,4h 45m,1 stop,No info,1,01,03,2019


In [700]:
df['Dep_hour'] = df['Dep_Time'].str.split(':').str[0]
df['Dep_minute'] = df['Dep_Time'].str.split(':').str[1]
df.drop('Dep_Time', axis=1, inplace=True)

df['Arrival_hour'] = df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[0]
df['Arrival_minute'] = df['Arrival_Time'].str.split(' ').str[0].str.split(':').str[1]
df.drop('Arrival_Time', axis=1, inplace=True)

df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,is_train,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_minute
0,IndiGo,Banglore,New Delhi,2h 50m,non-stop,No info,1,24,03,2019,22,20,01,10
1,Air India,Kolkata,Banglore,7h 25m,2 stops,No info,1,1,05,2019,05,50,13,15
2,Jet Airways,Delhi,Cochin,19h,2 stops,No info,1,9,06,2019,09,25,04,25
3,IndiGo,Kolkata,Banglore,5h 25m,1 stop,No info,1,12,05,2019,18,05,23,30
4,IndiGo,Banglore,New Delhi,4h 45m,1 stop,No info,1,01,03,2019,16,50,21,35


In [701]:
# Fix duration parsing with better logic
def extract_duration_hours(duration_str):
    try:
        if 'h' in duration_str:
            return int(duration_str.split('h')[0])
        else:
            return 0
    except:
        return 0

def extract_duration_minutes(duration_str):
    try:
        if 'm' in duration_str:
            parts = duration_str.split(' ')
            for part in parts:
                if 'm' in part:
                    return int(part.replace('m', ''))
            return 0
        else:
            return 0
    except:
        return 0

# Apply the functions
df['Duration_hour'] = df['Duration'].apply(extract_duration_hours)
df['Duration_minute'] = df['Duration'].apply(extract_duration_minutes)
df.drop('Duration', axis=1, inplace=True)
df['Duration_minutes'] = df['Duration_hour']*60 + df['Duration_minute']
df.drop(['Duration_hour','Duration_minute'], axis=1, inplace=True)

df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,is_train,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_minute,Duration_minutes
0,IndiGo,Banglore,New Delhi,non-stop,No info,1,24,03,2019,22,20,01,10,170
1,Air India,Kolkata,Banglore,2 stops,No info,1,1,05,2019,05,50,13,15,445
2,Jet Airways,Delhi,Cochin,2 stops,No info,1,9,06,2019,09,25,04,25,1140
3,IndiGo,Kolkata,Banglore,1 stop,No info,1,12,05,2019,18,05,23,30,325
4,IndiGo,Banglore,New Delhi,1 stop,No info,1,01,03,2019,16,50,21,35,285


In [702]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           13354 non-null  object
 1   Source            13354 non-null  object
 2   Destination       13354 non-null  object
 3   Total_Stops       13353 non-null  object
 4   Additional_Info   13354 non-null  object
 5   is_train          13354 non-null  int64 
 6   Date              13354 non-null  object
 7   Month             13354 non-null  object
 8   Year              13354 non-null  object
 9   Dep_hour          13354 non-null  object
 10  Dep_minute        13354 non-null  object
 11  Arrival_hour      13354 non-null  object
 12  Arrival_minute    13354 non-null  object
 13  Duration_minutes  13354 non-null  int64 
dtypes: int64(2), object(12)
memory usage: 1.4+ MB


In [703]:
df[['Date','Month','Year','Dep_hour','Dep_minute','Arrival_hour','Arrival_minute','Duration_minutes']].isnull().sum()

Date                0
Month               0
Year                0
Dep_hour            0
Dep_minute          0
Arrival_hour        0
Arrival_minute      0
Duration_minutes    0
dtype: int64

In [704]:
df[['Date','Month','Year','Dep_hour','Dep_minute','Arrival_hour','Arrival_minute']] = df[['Date','Month','Year','Dep_hour','Dep_minute','Arrival_hour','Arrival_minute']].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13354 entries, 0 to 13353
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Airline           13354 non-null  object
 1   Source            13354 non-null  object
 2   Destination       13354 non-null  object
 3   Total_Stops       13353 non-null  object
 4   Additional_Info   13354 non-null  object
 5   is_train          13354 non-null  int64 
 6   Date              13354 non-null  int64 
 7   Month             13354 non-null  int64 
 8   Year              13354 non-null  int64 
 9   Dep_hour          13354 non-null  int64 
 10  Dep_minute        13354 non-null  int64 
 11  Arrival_hour      13354 non-null  int64 
 12  Arrival_minute    13354 non-null  int64 
 13  Duration_minutes  13354 non-null  int64 
dtypes: int64(9), object(5)
memory usage: 1.4+ MB


In [705]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,is_train,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_minute,Duration_minutes
0,IndiGo,Banglore,New Delhi,non-stop,No info,1,24,3,2019,22,20,1,10,170
1,Air India,Kolkata,Banglore,2 stops,No info,1,1,5,2019,5,50,13,15,445
2,Jet Airways,Delhi,Cochin,2 stops,No info,1,9,6,2019,9,25,4,25,1140
3,IndiGo,Kolkata,Banglore,1 stop,No info,1,12,5,2019,18,5,23,30,325
4,IndiGo,Banglore,New Delhi,1 stop,No info,1,1,3,2019,16,50,21,35,285


In [706]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [707]:
df['Total_Stops'] = df['Total_Stops'].map({'non-stop':0, '2 stops':2, '1 stop':1, '3 stops':3, '4 stops':4})
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,is_train,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_minute,Duration_minutes
0,IndiGo,Banglore,New Delhi,0.0,No info,1,24,3,2019,22,20,1,10,170
1,Air India,Kolkata,Banglore,2.0,No info,1,1,5,2019,5,50,13,15,445
2,Jet Airways,Delhi,Cochin,2.0,No info,1,9,6,2019,9,25,4,25,1140
3,IndiGo,Kolkata,Banglore,1.0,No info,1,12,5,2019,18,5,23,30,325
4,IndiGo,Banglore,New Delhi,1.0,No info,1,1,3,2019,16,50,21,35,285


In [708]:
df['Total_Stops'].fillna(df['Total_Stops'].mode()[0],inplace=True)

C:\Users\jishn\AppData\Local\Temp\ipykernel_26488\2037983482.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Total_Stops'].fillna(df['Total_Stops'].mode()[0],inplace=True)


In [709]:
df.isnull().sum()

Airline             0
Source              0
Destination         0
Total_Stops         0
Additional_Info     0
is_train            0
Date                0
Month               0
Year                0
Dep_hour            0
Dep_minute          0
Arrival_hour        0
Arrival_minute      0
Duration_minutes    0
dtype: int64

In [710]:
print(df['Airline'].unique(),df['Source'].unique(),df['Destination'].unique())


['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet'] ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai'] ['New Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Delhi' 'Hyderabad']


In [711]:
%pip install scikit-learn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Additional_Info'] = le.fit_transform(df['Additional_Info'])

cols_to_encode = ['Airline', 'Source', 'Destination']

ohe = OneHotEncoder(sparse_output=False, drop=None)
encoded = ohe.fit_transform(df[cols_to_encode])
encoded_cols = ohe.get_feature_names_out(cols_to_encode)
encoded_df = pd.DataFrame(encoded, columns=encoded_cols, index=df.index)
df = pd.concat([df.drop(cols_to_encode, axis=1), encoded_df], axis=1)

df.head()


Note: you may need to restart the kernel to use updated packages.


,Total_Stops,Additional_Info,is_train,Date,Month,Year,Dep_hour,Dep_minute,Arrival_hour,Arrival_minute,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.0,0,1,24,3,2019,22,20,1,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2.0,0,1,1,5,2019,5,50,13,15,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0,1,9,6,2019,9,25,4,25,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0,1,12,5,2019,18,5,23,30,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0,1,1,3,2019,16,50,21,35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [712]:
from sklearn.preprocessing import StandardScaler
import pandas as pd

train_clean = df[df['is_train'] == 1].drop('is_train', axis=1)
test_clean = df[df['is_train'] == 0].drop('is_train', axis=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(train_clean)
X_test = scaler.transform(test_clean)

train_clean_scaled = pd.DataFrame(X_train, columns=train_clean.columns)
train_clean_scaled['Price'] = y.values

test_clean_scaled = pd.DataFrame(X_test, columns=test_clean.columns)

train_clean_scaled.to_excel("Cleaned_Train.xlsx", index=False)
test_clean_scaled.to_excel("Cleaned_Test.xlsx", index=False)

print("Cleaned and scaled train/test datasets saved successfully!!")


Cleaned and scaled train/test datasets saved successfully!!
